### 学习目标

- 可以基于报告内容和形式组织报告自动化脚本的文档结构
- 在编写不同报告的数据分析模块时产生技术积累
- 对分析模块进行泛用话处理，以便于积累更多通用模块

在7.1的数据读取章节我们有出现一个把函数分类放置的处理：
把str2num放入tools模块，数据读取放入datapipeline模块，在此我们同样是要进行类似的工作，把不同的函数进行分类处理

### 1. 文件组织形式
一般来说自动化报告的组织形式如下：

格式：
- ***文件夹*** ： 加粗倾斜
- *数据文件* ： 倾斜
- **执行模块** ： 加粗
- 未分类文件 ： 无


- ***report*** 自动化报告的工作目录
    - **\__init__.py**
    - main.ipynb 文件的主执行和调试文件，显示内容和结果，及进行自定义
    - ***data*** 保存数据文件 Save data to be used or generated
        - *data.csv*
    - ***image*** 存放生成的图片文件 Save image
    - ***report*** 存放生成的报告
    - ***template*** 存放报告生成需要的模板
        - template.html 生成HTML形式报告所用模板
        - template.tex 生成PDF报告需要的模板，这里我们使用python先生成Tex，然后Tex再生成PDF
        
        模板的后缀只影响用编辑器打开模板时的格式，不影响Jinja2生成模板，因为这些内容都是使用python来进行的文件读取
    - **tools.py** 保存工具型函数，可多次复用并且功能单一
    - **datapipeline.py** 数据读取和预处理，可从中直接取出清洗后数据
    - **ImageFactory.py** 图片工具包，可以把设置完毕的图片模板保存在此处，方便多次使用
    - **models.py** 定义文档结构或者数据结构
    - **configs.py** 设置报告的参数，比如作者，日期，数据源等等，每次使用时可以仅仅修改设置便生成不同报告
    - readme.txt 报告的说明文件
    
    类似于此处的文档结构说明可加入之后的readme文件中，具体说明文件目录中各文件功能，还可以更加详细地说明内含方法
    - requirements.txt 关于报告生成所用python环境中所用包及其版本的规定，为之后使用环境配置方便，直接`pip install -r requirements.txt`安装
    

### 2. 报告产生流程
作报告，原材料就是数据，数据分析就是一个个加工步骤，且产出的报告或者说结论就是最终的产品，接下来要介绍数据是怎么在这个框架中变成产品的

- 数据获取后存入***data***，数据获取方式业务之间各有不同，再次不做说明，爬虫和数据库的教程网上应该有做好的可用
- **datapipeline.py**中的模块可以对***data***中的数据自动进行数据清洗工作，可能会需要**configs.py**中的设置
- **models.py**中已经定义好的数据结构可以对从**datapipeline.py**导入的清洗好的数据进行进一步处理
- main.ipynb
  
  从**configs.py**导入设置
  
  从**models.py**导入文档结构和数据结构

  从**ImageFactory.py**导入image模板

  从**datapipeline.py**导入处理好的数据
  
  从**tools.py**导入数据分析工具
  
  分析数据
  
  生成图片
  
  添加说明和人工分析
  
  保存到document类
  
  使用***template***中的模板渲染document生成report 
  

### 3. 工作流程

1. 把需要使用的数据文件放入data文件夹中，
数据分两种，一种是生成报告使用的支持数据，更新变动很慢，但每次作报告都需要使用，比如行业均值方差啊，拟合结果的参数啊，这种建议建立数据库使用sql管理；一种是和报告一一对应的，这种放在data文件夹中很合适。
2. 对configs.py进行设置
3. 执行main.py在ipynb中显示图片和文字结果，若是半自动则进行人工文字补充
4. 执行main.py中导出report.html

### 4. 模块化

依旧使用7.1数据读取章节中的例子，当时是很粗略地说“把str2num放入tools模块，数据读取放入datapipeline模块”，但是具体操作的时候仅仅是复制粘贴是不够的，为了方便复用和泛用，还需要加下注释。

In [8]:
def str2num(string):
    """
    str2num(string)
    
    Get number for a string.
    
    Parameters
    ----------
    string : a string with the format like '$2.1', '$1, 333' or '&4,3'
    
    Returns
    -------
    out : float
    
    Examples
    --------
    >>> str2num('$2.3')
    2.3
    """
    if not isinstance(string, str):
        string = str(string)
    string = string.replace(',','')
    regular_expression = '\d+\.?\d*'
    pattern = re.compile(regular_expression)
    match = pattern.search(string)
    if match:
        return float(match.group())
    else:
        return float('nan')

可以使用`str2num`查看效果

In [11]:
?str2num

而数据读取可以简单地按照数据读取顺序放入文件

In [14]:
import pandas as pd
converters = {'Price':str2num,
              'Rank':str2num,
              'Rating':str2num,
              'Sales':str2num,
              'Revenue':str2num,
              'Reviews':str2num
             }
try:
    data = pd.read_csv('data.csv', converters=converters, header=7, index_col=0)
except BaseException as e:
    print(e)